In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd

# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [44]:
from PIL import Image

In [46]:
dir_img_1 = '/kaggle/input/traindataset/LUN_DataSet/train_url1'
dir_img_2 = '/kaggle/input/traindataset/LUN_DataSet/train_url2'

In [52]:
# Specify the path to the saved model directory
model_path = '/kaggle/input/triplet-cosine-dist/best_model'
# with tpu_strategy.scope():
# Load the saved model
loaded_model = tf.keras.models.load_model(model_path)

feat_vect_mob_net = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5", trainable=False)
feat_vect_mob_net.set_weights(loaded_model.layers[0].get_weights())

In [3]:
def cosine_similarity_batched(x1, x2):
    # Compute L2-norm of each vector
    x1_norm = tf.linalg.norm(x1, axis=1, keepdims=True)
    x2_norm = tf.linalg.norm(x2, axis=1, keepdims=True)

    # Compute dot product between vectors
    dot_product = tf.reduce_sum(x1 * x2, axis=1, keepdims=True)

    # Compute cosine similarity
    cosine_similarity = dot_product / (x1_norm * x2_norm)

    return cosine_similarity

In [4]:
def custom_f1_score(y_true, y_pred):
    # Calculate true positives, false positives, and false negatives
    TP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FN = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), dtype=tf.float32), axis=0)

    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

TRAINING (select threshold)

In [5]:
train_data = pd.read_csv('/kaggle/input/new-data-lun-csv/new_data_lun.csv')

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np
import os


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size=32, image_size=(224, 224)):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images1 = []
        batch_images2 = []
        batch_labels = []

        for _, row in batch_df.iterrows():
            img1_url, img2_url, label = row['image_url1'], row['image_url2'], row['is_same']
            image1 = Image.open(os.path.join(dir_img_1, img1_url))
            image2 = Image.open(os.path.join(dir_img_2, img2_url))
            if image1.mode == "L" or image1.mode == "RGBA":
                image1 = image1.convert("RGB")
            if image2.mode == "L" or image2.mode == "RGBA":
                image2 = image2.convert("RGB")
            
            image1 = image1.resize((224, 224))  
            image2 = image2.resize((224, 224))  
                
            image1 = tf.convert_to_tensor(image1)
            image2 = tf.convert_to_tensor(image2)

            batch_images1.append(image1)
            batch_images2.append(image2)
            batch_labels.append(label)

        batch_images1 = np.array(batch_images1) / 255.0  # Normalize images
        batch_images2 = np.array(batch_images2) / 255.0  # Normalize images
        batch_labels = np.array(batch_labels)
        return (batch_images1, batch_images2, batch_labels)

    def on_epoch_end(self):
        self.dataframe = self.dataframe.sample(frac=1)

In [7]:
split_index = int(len(train_data) * 0.8)
train_dataframe = train_data[:split_index]
val_dataframe = train_data[split_index:]

In [36]:
train_data_generator = DataGenerator(train_dataframe, batch_size=32)
val_data_generator = DataGenerator(val_dataframe, batch_size=32)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

In [57]:
class MyCustomModel(tf.keras.Model):
    def __init__(self, threshold):
        super(MyCustomModel, self).__init__()
        self.threshold = threshold
        self.base_model = feat_vect_mob_net

    def call(self, inputs):
        image1, image2 = inputs
        # Get embeddings from base models
        embedding_1 = self.base_model(image1)
        embedding_2 = self.base_model(image2)

        # Compute cosine similarity
        cosine_similarity = cosine_similarity_batched(embedding_1, embedding_2)
        output = tf.cast(cosine_similarity > self.threshold, tf.int32)
        return output

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
# Clear any logs from previous runs
!rm -rf /logs/

In [21]:
from tensorboard.plugins.hparams import api as hp

In [39]:
HP_THRESHOLD = hp.HParam('threshold', hp.RealInterval(0.7, 0.9))

METRIC_ACCURACY = 'f1'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_THRESHOLD],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='F1')],
  )

In [59]:
from sklearn.metrics import f1_score

def test_model(hparams):
    model = MyCustomModel(hparams[HP_THRESHOLD])
    model.compile(optimizer='sgd',
                loss='binary_crossentropy',
                metrics=['f1'])
    predictions = []
    labels = []
    for inputs1, inputs2, labels in train_dataset:
        outputs = model([inputs1, inputs2])
        outputs = tf.squeeze(outputs)
        outputs = outputs.numpy()
        outputs = outputs.tolist()
        predictions += outputs
        labels = tf.squeeze(labels)
        labels = labels.numpy()
        labels = labels.tolist()
        labels += labels
    return f1_score(labels, predictions, average='weighted')

In [55]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  
        accuracy = test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0
for threshold in (HP_THRESHOLD.domain.min_value, HP_THRESHOLD.domain.max_value):  
    hparams = {
              HP_THRESHOLD: threshold,
          }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'threshold': 0.7}


In [ ]:
%tensorboard --logdir logs/hparam_tuning

TEST

In [ ]:
test_data = pd.read_csv('/kaggle/input/download-test-data-lun/test_data_lun/new_test_data_lun.csv')

In [ ]:
dir_test_1 = '/kaggle/input/download-test-data-lun/test_data_lun/image_url1'
dir_test_2 = '/kaggle/input/download-test-data-lun/test_data_lun/image_url2'

In [ ]:
test_submission = pd.read_csv('/kaggle/input/copy-of-csc-hackathon-2023-lunua-task-2/test-data.csv')
test_data = pd.read_csv('/kaggle/input/download-test-data-lun/test_data_lun/new_test_data_lun.csv')
test_data["ID"] = list(test_submission["ID"].values)
test_data.head()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np
import os


class TestDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size=32, image_size=(224, 224)):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images1 = []
        batch_images2 = []

        for _, row in batch_df.iterrows():
            img1_url, img2_url = row['image_url1'], row['image_url2']
            if img1_url == "empty" or img2_url == "empty":
                image1 = tf.zeros((224, 224, 3))
                image2 = tf.zeros((224, 224, 3))
            else:
                image1 = Image.open(os.path.join(dir_test_1, img1_url))
                image2 = Image.open(os.path.join(dir_test_2, img2_url))
                if image1.mode == "L" or image1.mode == "RGBA":
                    image1 = image1.convert("RGB")
                if image2.mode == "L" or image2.mode == "RGBA":
                    image2 = image2.convert("RGB")

                image1 = image1.resize((224, 224))  
                image2 = image2.resize((224, 224))  

                image1 = tf.convert_to_tensor(image1)
                image2 = tf.convert_to_tensor(image2)

            batch_images1.append(image1)
            batch_images2.append(image2)

        batch_images1 = np.array(batch_images1) / 255.0  # Normalize images
        batch_images2 = np.array(batch_images2) / 255.0  # Normalize images
        return batch_images1, batch_images2

In [ ]:
from PIL import Image

In [ ]:
last_index = 22000
prev_idx = 0
next_idx = 0
predictions = []
    
for i in range(0, len(test_data), 1000):
    print(i)
    if i != 0:
        next_idx = i
        test_df = test_data[prev_idx: next_idx]
        test_data_generator = TestDataGenerator(test_df, batch_size=32)

        test_dataset = tf.data.Dataset.from_generator(
            lambda: test_data_generator,
            output_signature=(
                tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                 tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )
        prev_idx = i
        for inputs1, inputs2 in test_data_generator:
            outputs = model([inputs1, inputs1, best_threshold])
            outputs = tf.squeeze(outputs)
            outputs = outputs.numpy()
            outputs = outputs.tolist()
            print(outputs)
            predictions += outputs

In [ ]:
test_df = test_data[22000:]
test_data_generator = TestDataGenerator(test_df, batch_size=32)
test_dataset = tf.data.Dataset.from_generator(
            lambda: test_data_generator,
            output_signature=(
                tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                 tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )
prev_idx = i
for inputs1, inputs2 in test_data_generator:
    outputs = model([inputs1, inputs1, best_threshold])
    outputs = tf.squeeze(outputs)
    outputs = outputs.numpy()
    outputs = outputs.tolist()
    predictions += outputs

In [ ]:
ids = list(test_data["ID"].values)
submission = pd.DataFrame()
submission["ID"] = ids
submission["is_same"] = predictions

In [ ]:
submission.to_csv('submission.csv', index=False)